### Here I create BED files for each species comparison.

In [3]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/get_best_hits")

library(pafr)
library(dplyr)

options(scipen = 100, digits = 10)

myo_meta <- read.csv("../../../data/myotis_meta.csv")
myo_meta

Loading required package: ggplot2




Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




Field_Name,Abbr
<chr>,<chr>
Myotis-Auriculus,mMyoAui
Myotis-Californicus,mMyoCai
Myotis-Evotis,mMyoEvo
Myotis-Lucifugus,mMyoLuc
Myotis-Occultus,mMyoOcc
Myotis-Thysanodes,mMyoThy
Myotis-Velifer,mMyoVel
Myotis-Volans,mMyoVol
Myotis-Yumanensis,mMyoYum


In [5]:
for (i in seq_along(myo_meta$Abbr)) {
  
  for (j in seq_along(myo_meta$Abbr)) {

    if (i < j) {
    
      working_paf1 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[j])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[i]))
      )))

      working_paf2 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[i])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[j]))
      )))

      ## Filter sequences with mapq < 10:
      ## Sniffles mapping quality threshold to 10
      ## Ref: https://arxiv.org/pdf/2108.03515.pdf
      
      working_paf1 <- working_paf1[working_paf1$mapq >= 10,]
      working_paf2 <- working_paf2[working_paf2$mapq >= 10,]

      ## Ensure ID is unique:
      
      working_paf1$Id <- with(working_paf1, paste(qstart, qend, tstart, tend, id, sep = "_"))      
      working_paf2$Id <- with(working_paf2, paste(qstart, qend, tstart, tend, id, sep = "_"))      

      ## Species 1 positions:

      spec1_query <- working_paf1 %>%
        dplyr::select(
          qname, qstart, qend, strand,
          nmatch, alen, mapq, id, Id
        )

      spec1_target <- working_paf2 %>%
        dplyr::select(
          tname, tstart, tend,
          nmatch, alen, mapq, id, Id
        )

      write.table(
        spec1_query,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_positions_query.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

      write.table(
        spec1_target,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_positions_target.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

      ## Species 2 positions:

      spec2_query <- working_paf2 %>%
        dplyr::select(
          qname, qstart, qend, strand,
          nmatch, alen, mapq, id, Id
        )

      spec2_target <- working_paf1 %>%
        dplyr::select(
          tname, tstart, tend,
          nmatch, alen, mapq, id, Id
        )

      write.table(
        spec2_query,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_positions_query.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

      write.table(
        spec2_target,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_positions_target.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

    }
    
  }
  
}


In [63]:
working_paf1 %>%
  arrange(tstart) %>%
  filter(qname == "mMyoCai1.0.SUPER__1" & tname == "mMyoAui1.0.SUPER__1") %>%
  head()

working_paf2 %>%
  arrange(qstart) %>%
  filter(tname == "mMyoCai1.0.SUPER__1" & qname == "mMyoAui1.0.SUPER__1") %>%
  head()

,qname,qlen,qstart,qend,strand,tname,tlen,tstart,tend,nmatch,alen,mapq,id
,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,mMyoCai1.0.SUPER__1,234393516,233250000,234393516,-,mMyoAui1.0.SUPER__1,234041302,2095881,3236853,10723,1143516,14,95.9830
2,mMyoCai1.0.SUPER__1,234393516,231865000,231945000,+,mMyoAui1.0.SUPER__1,234041302,3257751,3346903,851,89152,8,82.8577
3,mMyoCai1.0.SUPER__1,234393516,231910000,231990000,+,mMyoAui1.0.SUPER__1,234041302,3414111,3498921,799,84810,7,80.0921
4,mMyoCai1.0.SUPER__1,234393516,232195000,232890000,+,mMyoAui1.0.SUPER__1,234041302,3453707,4165946,6964,712239,15,96.5665
5,mMyoCai1.0.SUPER__1,234393516,230495000,231685000,+,mMyoAui1.0.SUPER__1,234041302,4289627,5503974,11638,1214347,14,95.9911
6,mMyoCai1.0.SUPER__1,234393516,225990000,230280000,-,mMyoAui1.0.SUPER__1,234041302,5775850,10061158,41675,4290000,15,97.0659


,qname,qlen,qstart,qend,strand,tname,tlen,tstart,tend,nmatch,alen,mapq,id
,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,mMyoAui1.0.SUPER__1,234041302,2095000,3225000,-,mMyoCai1.0.SUPER__1,234393516,233259906,234393409,10622,1133503,14,95.9816
2,mMyoAui1.0.SUPER__1,234041302,3520000,4110000,+,mMyoCai1.0.SUPER__1,234393516,232254999,232839473,5717,590000,15,96.5526
3,mMyoAui1.0.SUPER__1,234041302,4290000,5550000,+,mMyoCai1.0.SUPER__1,234393516,230495325,231721227,12146,1260000,14,95.8486
4,mMyoAui1.0.SUPER__1,234041302,5775000,10090000,-,mMyoCai1.0.SUPER__1,234393516,225981652,230279117,42157,4315000,15,97.0425
5,mMyoAui1.0.SUPER__1,234041302,10070000,11200000,+,mMyoCai1.0.SUPER__1,234393516,224963021,226028428,11050,1130000,15,96.5647
6,mMyoAui1.0.SUPER__1,234041302,11200000,11250000,+,mMyoCai1.0.SUPER__1,234393516,225982113,226027391,489,50000,8,82.4627
